In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import imodelsx.process_results
from collections import defaultdict
import numpy as np
import viz
import sys
import notebook_helper
from matplotlib.backends.backend_pdf import PdfPages
sys.path.append('../experiments/')
# results_dir = '/home/chansingh/mntv1/tree-prompt/feb25'
# results_dir = '/home/chansingh/mntv1/tree-prompt/mar17'
results_dir = '/home/chansingh/mntv1/tree-prompt/mar18'

# r = imodelsx.process_results.get_results_df(results_dir)
# experiment_filename = '../experiments/01_fit.py'
# r = imodelsx.process_results.fill_missing_args_with_default(r, experiment_filename)
# r['mean_llm_calls'] = notebook_helper.add_mean_llm_calls(r)
# r = r[r.model_name != 'manual_rf']
# r.to_pickle('../results/tprompt_results.pkl')
r = pd.read_pickle('../results/tprompt_results.pkl')
ravg = r[r.seed == 1]

In [ ]:
groupings = ['checkpoint', 'prompt_source', 'verbalizer_num'] #, 'binary_classification']
for k, rcurve in ravg.groupby(groupings):
    rcurve = rcurve.sort_values(by=['dataset_name', 'model_name', 'mean_llm_calls'])

    metric = 'roc_auc_test'
    # metric = 'accuracy'
    x = 'mean_llm_calls'                # x = 'num_prompts'

    viz.plot_perf_curves_individual(rcurve, x=x, xlim=15, metric=metric)
    x = rcurve.iloc[0]
    plt.suptitle(f'''checkpoint={viz.CHECKPOINTS_RENAME_DICT[x.checkpoint]} prompt_source={x.prompt_source} verb={viz.VERBS_RENAME_DICT[x.verbalizer_num]}''', fontsize='xx-small')
    # plt.tight_layout()
    # plt.show()

    # save a couple
    row = rcurve.iloc[0]
    if row.checkpoint == 'EleutherAI/gpt-j-6B' and row.prompt_source == 'data_demonstrations' and row.verbalizer_num == 0:
        plt.suptitle(f'Model: {viz.CHECKPOINTS_RENAME_DICT[x.checkpoint]}, Prompts: {viz.SOURCE_RENAME_DICT.get(x.prompt_source, x.prompt_source)}', fontsize='x-small')
        plt.savefig('../results/figs/perf_curves_gptj.pdf', bbox_inches='tight')
    if row.checkpoint == 'gpt2' and row.prompt_source == 'manual' and row.verbalizer_num == 0:
        plt.suptitle(f'Model: {viz.CHECKPOINTS_RENAME_DICT[x.checkpoint]}, Prompts: {viz.SOURCE_RENAME_DICT.get(x.prompt_source, x.prompt_source)}', fontsize='x-small')
        plt.savefig('../results/figs/perf_curves_gpt2.pdf', bbox_inches='tight')

viz.save_figs_to_single_pdf("all_results.pdf")

## Table of results

In [ ]:
rd = ravg
groupings2 = ['checkpoint', 'prompt_source', 'verbalizer_num', 'dataset_name', 'model_name']

dset_names = rd.dataset_name.unique().tolist()
mets = defaultdict(list)
for k, rcurve in rd.groupby(groupings2):    
    # rcurve = rcurve.sort_values(by='accuracy_cv', ascending=False)
    # metadata
    for kk in groupings2:
        mets[kk].append(rcurve.iloc[0][kk])
    mets['roc_auc<=5'].append(rcurve[rcurve['mean_llm_calls'] <= 5]['roc_auc_test'].max())

df = pd.DataFrame.from_dict(mets)
df['dataset_name'] = df['dataset_name'].apply(viz.DSETS_RENAME_DICT_ABBR.get)
df['model_name'] = df['model_name'].apply(viz.MODELS_RENAME_DICT.get)
df['checkpoint'] = df['checkpoint'].apply(viz.CHECKPOINTS_RENAME_DICT.get)
df['prompt_source'] = df['prompt_source'].apply(viz.SOURCE_RENAME_DICT.get)
df['verbalizer_num'] = df['verbalizer_num'].apply(viz.VERBS_RENAME_DICT.get)
# df['prompt_source'] = df['prompt_source'].apply(viz.PROMPT_SOURCES_RENAME_DICT.get)
# print(df.style.hide().to_latex())
dfp = df.pivot_table(index=['dataset_name', 'checkpoint', 'prompt_source', 'verbalizer_num'], columns=['model_name'], values='roc_auc<=5').round(2)
print(dfp.style.format(precision=2, escape=True).to_latex(hrules=True).replace('NaN', '----').replace('_', ' '))

In [ ]:
print(
    "mean improvement",
    np.mean(dfp["TreePrompt"] - dfp["Ensemble (Boosting)"]),
    np.mean(dfp["TreePrompt"] - dfp["Ensemble (Greedy)"]),
)

### Choose best results

In [ ]:
# apply cross validation
ravg_cv = (
    ravg
    .sort_values(by='accuracy_cv', ascending=False)
    .groupby(by=['dataset_name', 'model_name', 'checkpoint'])
    .first()
    .reset_index()
)
ravg_cv

In [82]:
rtab = ravg_cv[['dataset_name', 'model_name', 'checkpoint', 'accuracy_cv', 'accuracy_test']]
rtab = rtab.pivot_table(index=['checkpoint', 'model_name'], columns=['dataset_name'], values=['accuracy_test']).round(3)
rtab.columns = rtab.columns.get_level_values(1)

rtab

dataset_name                         emotion  financial_phrasebank   imdb   
checkpoint          model_name                                              
EleutherAI/gpt-j-6B manual_boosting    0.563                 0.746  0.942  \
                    manual_ensemble    0.491                 0.692  0.935   
                    manual_gbdt        0.587                 0.802  0.945   
                    manual_tree        0.562                 0.788  0.942   
gpt2                manual_boosting    0.558                 0.701  0.854   
                    manual_ensemble    0.507                 0.618  0.845   
                    manual_gbdt        0.555                 0.714  0.858   
                    manual_tree        0.544                 0.712  0.851   

dataset_name                         rotten_tomatoes   sst2  
checkpoint          model_name                               
EleutherAI/gpt-j-6B manual_boosting            0.902  0.916  
                    manual_ensemble            0.886  0.905  
                    manual_gbdt                0.905  0.923  
                    manual_tree                0.899  0.915  
gpt2                manual_boosting            0.777  0.811  
                    manual_ensemble            0.772  0.784  
                    manual_gbdt                0.775  0.808  
                    manual_tree                0.776  0.790

In [86]:
print(rtab.style.format(precision=2).to_latex())

\begin{tabular}{llrrrrr}
 & dataset_name & emotion & financial_phrasebank & imdb & rotten_tomatoes & sst2 \\
checkpoint & model_name &  &  &  &  &  \\
\multirow[c]{4}{*}{EleutherAI/gpt-j-6B} & manual_boosting & 0.56 & 0.75 & 0.94 & 0.90 & 0.92 \\
 & manual_ensemble & 0.49 & 0.69 & 0.94 & 0.89 & 0.91 \\
 & manual_gbdt & 0.59 & 0.80 & 0.94 & 0.91 & 0.92 \\
 & manual_tree & 0.56 & 0.79 & 0.94 & 0.90 & 0.92 \\
\multirow[c]{4}{*}{gpt2} & manual_boosting & 0.56 & 0.70 & 0.85 & 0.78 & 0.81 \\
 & manual_ensemble & 0.51 & 0.62 & 0.84 & 0.77 & 0.78 \\
 & manual_gbdt & 0.56 & 0.71 & 0.86 & 0.78 & 0.81 \\
 & manual_tree & 0.54 & 0.71 & 0.85 & 0.78 & 0.79 \\
\end{tabular}

